In [1]:
import os
import json
import base64
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from uuid import uuid4
from random import randint
from utils.get_openai_api_key import get_test_key
from utils.get_postgres_connection import _conn_open
from utils.load_json import load_json
from rich.console import Console
import psycopg2

In [2]:
print("Connecting to PostgreSQL...")
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
)

cur = conn.cursor()

print("Successfully connected to PostgreSQL.")

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.


In [3]:
console = Console()

In [4]:
OPENAI_API_KEY = get_test_key()
# Set up your OpenAI API key
openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
console.print(OPENAI_API_KEY[:12])

Success: .env file found with some environment variables

sk-proj-SztC

In [5]:
def get_embedding(text, model="text-embedding-3-small"):

    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [6]:
def decode_image(encoded_image_data, image_fimename):
    decoded_image_data = base64.b64decode(encoded_image_data)
    file_name_path = f"./{image_fimename}"
    with open(file_name_path, "wb") as new_image_file:
        new_image_file.write(decoded_image_data)

In [7]:
FILENAME = "TMCB_43_2256640.pdf.json"

In [8]:
def embed_and_load(filename=FILENAME):

    json_data = load_json(FILENAME)
    print(f"Number of items: {len(json_data)}")
    encoded_image_data = None
    file_id = str(uuid4())
    for i in range(len(json_data)):

        # if text_html then must be parsed for correct HTML quotes
        element_id = json_data[i]["element_id"]
        element_type = json_data[i]["type"].upper()
        if "parent_id" in json_data[i]:
            parent_id = json_data[i]["parent_id"]
        else:
            parent_id = ""
        page_number = json_data[i]["metadata"]["page_number"]
        filename = json_data[i]["metadata"]["filename"]
        element_text = json_data[i]["text"]
        if element_text is not None:
            element_embedding = get_embedding(element_text)
        else:
            element_embedding = ""
        if json_data[i]["type"] == "Image" or json_data[i]["type"] == "Table":
            console.print(
                f"[dark_orange]--------- {element_type.upper()} Item Number  {i + 1}--------[/]"
            )
            el_image_b64 = json_data[i]["metadata"]["image_base64"]
            # console.print(el_image_b64[:40])
        else:
            el_image_b64 = "NONE"

        sql = f"""
            INSERT INTO tbl_doc_elements (
                file_id, element_id, element_text, element_type, parent_id, page_number, image_base64, embedding)
            VALUES 
            ('{file_id}','{element_id}','{element_text}','{element_type}','{parent_id}', '{page_number}', '{el_image_b64}', '{element_embedding}')
            RETURNING id;
                """
        # console.print(sql)
        try:
            cur.execute(sql)
            row = cur.fetchone()
            id = row[0]
            console.print(f"{i} - ID is {id}\nFILE_ID: {file_id}")
            conn.commit()
        except Exception as e:
            console.print(e)

In [9]:
files = ["wef.pdf_6134.json"]

In [10]:
for file in files:
    print(file)
    FILENAME = file
    embed_and_load(filename=FILENAME)
    print(f"file {file} loaded")

wef.pdf_6134.json
Number of items: 618


ID is 079c8c24-ce4a-422d-905e-a10181b8c23c
FILE_ID: 9faf9e8e-157f-45c7-982b-94c7388801bf

ID is d6df18f5-b940-4c7f-97b8-c956e32ac055
FILE_ID: 9faf9e8e-157f-45c7-982b-94c7388801bf

ID is 67e42cd7-f5db-4262-a111-f0559c4d019c
FILE_ID: 9faf9e8e-157f-45c7-982b-94c7388801bf

ID is e23b1bae-02f6-4033-9a6d-78fb157d0ec8
FILE_ID: 9faf9e8e-157f-45c7-982b-94c7388801bf

...

ID is fec5bf8c-eaad-45a0-a20e-e3582bd039ba
FILE_ID: 9faf9e8e-157f-45c7-982b-94c7388801bf

ID is 5498012b-5f1a-4147-b360-3fc28a999875
FILE_ID: 9faf9e8e-157f-45c7-982b-94c7388801bf

ID is 589d1bfb-b9a2-45a2-b811-94d2083a34d8
FILE_ID: 9faf9e8e-157f-45c7-982b-94c7388801bf

ID is 3459cb13-b48c-4613-81a5-8fb873dd54ae
FILE_ID: 9faf9e8e-157f-45c7-982b-94c7388801bf

file wef.pdf_6134.json loaded


In [11]:
cur.close()
conn.close()